# Nussinov Algorithm Implementation

This notebook demonstrates the implementation of the Nussinov algorithm, a dynamic programming approach to predicting RNA secondary structures.
Each section is detailed to guide you through the algorithm's logic, from utility functions to backtracking and main execution.



## Importing Necessary Libraries and Constants

This section defines the constants and imports required for the Nussinov algorithm.


In [1]:
import os
import sys


## Utility Function: Validating Base Pairs

The `isBasePair` function determines whether two bases can form a valid RNA pair, such as A-U or G-C.
This function is crucial for the Nussinov algorithm to check pairing compatibility.


In [2]:
def isBasePair(i, j): #returns boolean for if its a pair or not
	pairs = {('A', 'U'), ('U', 'A'), ('C', 'G'), ('G', 'C')}
	return (i, j) in pairs


## Filling the Dynamic Programming Table

The `fill_nussinov` function implements the core dynamic programming logic of the Nussinov algorithm.
It computes the scoring matrix (`s`) by iterating over subsequences and considering all possible pairings and bifurcations.


In [3]:
def fill_nussinov(sequence):
    length = len(sequence)
    # Initialize the DP table
    dp_table = [[0 for _ in range(length)] for _ in range(length)]

    # Fill the DP table
    for window_size in range(1, length):
        for start in range(length - window_size):
            end = start + window_size
            if end >= start:
                # Calculate scores for each option
                score_down = dp_table[start + 1][end]
                score_left = dp_table[start][end - 1]
                score_diagonal = dp_table[start + 1][end - 1] + isBasePair(sequence[start], sequence[end])
                score_bifurcation = max(
                    dp_table[start][k] + dp_table[k + 1][end] for k in range(start, end)
                )

                # Choose the maximum score
                dp_table[start][end] = max(score_down, score_left, score_diagonal, score_bifurcation)

    # The optimal score is at the top-right corner of the DP table
    optimal_score = dp_table[0][length - 1]
    return optimal_score, dp_table


## Backtracking to Find Optimal Pairing

The `traceback` function uses the scoring matrix to reconstruct the optimal RNA secondary structure.
It follows the traceback directions (`s`) and adds valid pairings to the `pairs` list.


In [4]:
def traceback(dp_table, sequence, base_pairs, start, end):
    if start < end:
        if dp_table[start][end] == dp_table[start + 1][end]:
            traceback(dp_table, sequence, base_pairs, start + 1, end)
        elif dp_table[start][end] == dp_table[start][end - 1]:
            traceback(dp_table, sequence, base_pairs, start, end - 1)
        elif dp_table[start][end] == dp_table[start + 1][end - 1] + 1:
            base_pairs.append((start, end))
            traceback(dp_table, sequence, base_pairs, start + 1, end - 1)
        else:
            for middle in range(start + 1, end - 1):
                if dp_table[start][end] == dp_table[start][middle] + dp_table[middle + 1][end]:
                    traceback(dp_table, sequence, base_pairs, middle + 1, end)
                    traceback(dp_table, sequence, base_pairs, start, middle)
                    break

    return base_pairs


## Converting Pairings to Dot-Bracket Notation

The `convert_pairs` function translates the pairing results into the dot-bracket notation, a standard format for RNA secondary structure representation.


In [5]:
def convert_to_dot_bracket(sequence, base_pairs):
    # Initialize a list of dots representing unpaired bases
    dot_bracket_notation = ["." for _ in range(len(sequence))]

    # Mark paired indices with parentheses
    for (start, end) in base_pairs:
        dot_bracket_notation[start] = "("
        dot_bracket_notation[end] = ")"

    # Convert the list to a string and return
    return "".join(dot_bracket_notation)


## Main Execution: Running the Nussinov Algorithm

This section defines a test sequence and applies the Nussinov algorithm to predict its secondary structure.
It prints the final dot-bracket representation and the filled DP table.


In [6]:
if __name__ == '__main__':
    seq = 'GGGAAAUCC'
    ans, table = fill_nussinov(seq)
    pairs = traceback(table, seq, [], 0, len(seq) - 1)
    res = convert_to_dot_bracket(seq, pairs)
    print(seq)
    print(res)
    for r in table:
      print(r)

GGGAAAUCC
.((..()))
[0, 0, 0, 0, 0, 0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 1, 2, 2]
[0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
if __name__ == '__main__':
    seq = 'GGUCCAC'
    ans, table = fill_nussinov(seq)
    pairs = traceback(table, seq, [], 0, len(seq) - 1)
    res = convert_to_dot_bracket(seq, pairs)
    print(seq)
    print(res)
    for r in table:
      print(r)

GGUCCAC
.((..))
[0, 0, 0, 1, 2, 2, 2]
[0, 0, 0, 1, 1, 1, 2]
[0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]


In [11]:
if __name__ == '__main__':
    seq = 'GCACGACG'
    ans, table = fill_nussinov(seq)
    pairs = traceback(table, seq, [], 0, len(seq) - 1)
    res = convert_to_dot_bracket(seq, pairs)
    print(seq)
    print(res)
    for r in table:
      print(r)

GCACGACG
().((.))
[0, 1, 1, 1, 2, 2, 2, 3]
[0, 0, 0, 0, 1, 1, 1, 2]
[0, 0, 0, 0, 1, 1, 1, 2]
[0, 0, 0, 0, 1, 1, 1, 2]
[0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
if __name__ == '__main__':
    seq = 'GCUCGGGUUCCCUAUUCAAGAGC'
    ans, table = fill_nussinov(seq)
    pairs = traceback(table, seq, [], 0, len(seq) - 1)
    res = convert_to_dot_bracket(seq, pairs)
    print(seq)
    print(res)
    for r in table:
      print(r)

GCUCGGGUUCCCUAUUCAAGAGC
(((((((..)))(()(.))))))
[0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, 6, 7, 8, 9, 9, 10]
[0, 0, 0, 0, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 6, 7, 8, 9, 9]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3, 3, 3, 4, 4, 4, 4, 5, 6, 7, 8, 8, 9]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3, 3, 3, 4, 4, 4, 4, 5, 6, 7, 7, 7, 8]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 4, 4, 4, 4, 5, 6, 6, 6, 6, 7]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 6, 6, 6, 7]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 5, 6, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 4, 5, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 4, 5, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 4, 4, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 3, 3, 3, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0